In [107]:
import os
import json
import glob
import csv
import math
import pandas as pd
from io import StringIO
from pathlib import Path

In [108]:
jars = glob.glob("../versions/*/*.jar")
libs = glob.glob("../versions/*/lib")
versions =  glob.glob("../versions/*")
versions.sort()
versions_f = [v.split("\\")[-1] for v in versions]

In [109]:
layer_ex = dict()
ckjm_ex = dict()
for j, l, v, vf in zip(jars, libs, versions, versions_f): 
    process = os.popen(f"java -jar {Path('../layer_extractor.jar')} {j}")
    # layer_ex[vf] = process.read()
    layers = pd.read_csv(StringIO(process.read()))
    process.close()
    if("5.5.13.3" in j):
        # read csv
        ckjm_vals = pd.read_csv(Path("../versions/5.5.13.3/ck-metrics-5.5.13.3.csv"))
    else:
        process = os.popen(f"java -jar {Path('../ckjm_adapted.jar')} {j} {l}")
        ckjm_vals = pd.read_csv(StringIO(process.read()))
    
    ckjm_ex[vf] = ckjm_vals.merge(layers, on='ClassName')
    process.close()

In [118]:
num_classes = [len(a) for a in ckjm_ex.values()]
num_classes

<generator object <genexpr> at 0x7f6711f7da20>


In [111]:
def fwbr_n(CBO, DIT, WMC, RFC, LCOM):
 return -(9.023 * math.log(CBO + 1) + \
 3.945 * math.log(DIT + 1) - \
 0.678 * math.log(WMC + 1) - \
 0.826 * math.log(RFC + 1) - \
 0.378 * math.log(LCOM + 1))

In [112]:
cols = ['ClassName', 'WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM', 'CA', 'NPM', 'Layer']
ckjm_dict = {k.split('/')[-1]:v for k, v in ckjm_ex.items()}

In [126]:
# print(ckjm_dict)
for x in ckjm_dict.keys():
    print(f"{x}, {len(ckjm_dict[x])}")

5.0.6, 566
5.1.3, 587
5.2.1, 678
5.3.4, 638
5.4.5, 933
5.5.13.3, 827


In [114]:
for version in ckjm_dict:
    print(f"versie {version}")
    for metric in ckjm_dict[version].drop('ClassName', axis=1):
        # print(metric)
        print(f"{ckjm_dict[version][metric].astype('float64').min()} - {ckjm_dict[version][metric].astype('float64').max()}")
        # print(version, metric)

versie 5.0.6
0.0 - 176.0
1.0 - 6.0
0.0 - 42.0
0.0 - 86.0
0.0 - 436.0
0.0 - 9082.0
0.0 - 151.0
0.0 - 158.0
0.0 - 13.0
versie 5.1.3
0.0 - 176.0
1.0 - 6.0
0.0 - 31.0
0.0 - 86.0
0.0 - 438.0
0.0 - 9533.0
0.0 - 150.0
0.0 - 158.0
0.0 - 12.0
versie 5.2.1
0.0 - 195.0
1.0 - 6.0
0.0 - 32.0
0.0 - 89.0
0.0 - 455.0
0.0 - 11261.0
0.0 - 160.0
0.0 - 164.0
0.0 - 12.0
versie 5.3.4
0.0 - 181.0
1.0 - 6.0
0.0 - 31.0
0.0 - 86.0
0.0 - 438.0
0.0 - 9533.0
0.0 - 157.0
0.0 - 163.0
0.0 - 12.0
versie 5.4.5
0.0 - 249.0
1.0 - 6.0
0.0 - 34.0
0.0 - 102.0
0.0 - 561.0
0.0 - 17152.0
0.0 - 188.0
0.0 - 211.0
0.0 - 18.0
versie 5.5.13.3
0.0 - 208.0
1.0 - 6.0
0.0 - 34.0
0.0 - 94.0
0.0 - 499.0
0.0 - 12393.0
0.0 - 182.0
0.0 - 170.0
0.0 - 17.0


In [127]:
def fwbr_n(CBO, DIT, WMC, RFC, LCOM):
 return -(9.023 * math.log(CBO + 1) + \
 3.945 * math.log(DIT + 1) - \
 0.678 * math.log(WMC + 1) - \
 0.826 * math.log(RFC + 1) - \
 0.378 * math.log(LCOM + 1))

def apply_fwbr_n(row):
    return fwbr_n(row['CBO'], row['DIT'], row['WMC'], row['RFC'], row['LCOM'])

def fwbr(CBO, DIT, WMC, RFC, LCOM, NOC):
 return -(8.753 * math.log(CBO + 1) + \
 2.505 * math.log(DIT + 1) - \
 1.922* math.log(WMC + 1) - \
 0.892 * math.log(RFC + 1) - \
 0.399 * math.log(LCOM + 1) - \
 1.080 * math.log(NOC + 1))

def apply_fwbr(row):
    return fwbr(row['CBO'], row['DIT'], row['WMC'], row['RFC'], row['LCOM'], row['NOC'])


for version in ckjm_dict:
    for metric in ['WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM', 'CA', 'NPM']:
        ckjm_dict[version][metric] = pd.to_numeric(ckjm_dict[version][metric])
    ckjm_dict[version]['fwbr_n'] = ckjm_dict[version].apply(lambda row: apply_fwbr_n(row), axis=1)
    ckjm_dict[version]['fwbr'] = ckjm_dict[version].apply(lambda row: apply_fwbr(row), axis=1)



In [132]:
for x in ckjm_dict.keys():
    print(ckjm_dict[x])

                                             ClassName  WMC  DIT  NOC  CBO  \
0                 com.itextpdf.text.ExceptionConverter   10    4    0    0   
1                 com.itextpdf.text.pdf.qrcode.Version   13    1    0    5   
2           com.itextpdf.text.pdf.codec.wmf.MetaObject    3    1    3    0   
3    com.itextpdf.text.pdf.codec.TiffWriter$FieldImage    1    2    0    1   
4    com.itextpdf.text.pdf.parser.LocationTextExtra...    8    1    0    6   
..                                                 ...  ...  ...  ...  ...   
561                com.itextpdf.text.pdf.ExtendedColor    6    2    5    1   
562  com.itextpdf.text.pdf.parser.PdfContentStreamP...    3    1    0    6   
563  com.itextpdf.text.pdf.PdfWriter$PdfBody$PdfCro...   10    1    0    1   
564               com.itextpdf.text.pdf.PdfContentByte  176    1    4   53   
565          com.itextpdf.text.pdf.codec.wmf.MetaState   36    1    0    7   

     RFC  LCOM  CA  NPM  layer     fwbr_n       fwbr  
0     21